# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. P. Garcia  ->  A. P. Garcia  |  ['A. P. Garcia']
J. Li  ->  J. Li  |  ['J. Li']
A. Frank  ->  A. Frank  |  ['A. Frank']
F. Walter  ->  F. Walter  |  ['F. Walter']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Liu  ->  J. Liu  |  ['J. Liu']


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
J. Baier  ->  J. Baier  |  ['J. Baier']
Arxiv has 95 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2508.16775


extracting tarball to tmp_2508.16775...

 done.


Found 88 bibliographic references in tmp_2508.16775/aa56452-25.bbl.
Retrieving document from  https://arxiv.org/e-print/2508.16781


...

extracting tarball to tmp_2508.17307... done.
Retrieving document from  https://arxiv.org/e-print/2508.18097


extracting tarball to tmp_2508.18097... done.
Retrieving document from  https://arxiv.org/e-print/2508.18126


extracting tarball to tmp_2508.18126...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2508.18126/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'acks' from 'tmp_2508.18126/acks.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors_arx' from 'tmp_2508.18126/authors_arx.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.16775-b31b1b.svg)](https://arxiv.org/abs/2508.16775) | **Characterising the short-orbital period X-ray transient Swift J1910.2-0546**  |
|| J. M. Corral-Santana, et al. -- incl., <mark>A. P. Garcia</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| *13 pages, 10 figures. Accepted for publication in A&A*|
|**Abstract**|            SwiftJ1910.2-0546 is a Galactic X-ray transient discovered during a bright outburst in 2012. We use time-series optical photometry and spectroscopy to estimate the orbital period, characterise the donor star, determine the interstellar extinction, distance, and system geometry, and constrain the component masses. Multi-site r-band and clear-filter light curves and WHT/ACAM spectra from the 2012 outburst are combined with time-series spectroscopy from GTC/OSIRIS and VLT/FORS2 in quiescence. Period searches are conducted using generalised Lomb-Scargle, phase-dispersion minimisation, and analysis-of-variance algorithms. Diffuse interstellar bands constrain E(B-V), while empirical correlations involving H$\alpha$ yield estimates of K2, q, and i. We detect a double-humped modulation with a period of $0.0941\pm0.0007$d ($2.26\pm0.02$h) during the outburst. Its morphology is consistent with an early superhump, suggesting that the true orbital period may be slightly shorter than 4.52h. The H$\alpha$ radial velocity curves do not yield a definitive orbital period. In quiescence, TiO bands indicate an M3-M3.5 donor contributing 70% of the red continuum. Diffuse interstellar bands give E(B-V)=$0.60\pm0.05$ and N_H=$(3.9\pm1.3)$x10$^{21}$cm$^{-2}$, placing the system at a distance of 2.8-4.0 kpc. The H$\alpha$ line width in quiescence (FWHM_0 =$990\pm45$km/s), via a FWHM-K_2 calibration, provides an estimate of K_2, while its double peaked profile gives q and i. Adopting the resulting K_2=$230\pm17$km/s and q=$0.032\pm0.010$, and two orbital period scenarios (2.25 and 4.50h), Monte Carlo sampling returns a compact object mass M_1=8-11M_sun and an inclination i=13-18 deg for plausible donor masses (M_2=0.25-0.35M_sun). We favour an orbital period of 4.5h. Further phase-resolved spectroscopy and photometry during quiescence are needed to better determine its fundamental parameters.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.16781-b31b1b.svg)](https://arxiv.org/abs/2508.16781) | **Soot Planets instead of Water Worlds**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-08-26*|
|*Comments*| *20 pages, 8 figures, submitted to ApJ Letters*|
|**Abstract**|            Some low-density exoplanets are thought to be water-rich worlds that formed beyond the snow line of their protoplanetary disc, possibly accreting coequal portions of rock and water. However, the compositions of bodies within the Solar System and the stability of volatile-rich solids in accretionary disks suggest that a planet rich in water should also acquire as much as 40% refractory organic carbon ("soot"). This would reduce the water mass fraction well below 50%, making the composition of these planets similar to those of Solar System comets. Here we show that soot-rich planets, with or without water, can account for the low average densities of exoplanets that were previously attributed to a binary combination of rock and water. Formed in locations beyond the soot and/or snow lines in disks, these planets are likely common in our galaxy and already observed by JWST. The surfaces and interiors of soot-rich planets will be influenced by the chemical and physical properties of carbonaceous phases, and the atmospheres of such planets may contain plentiful methane and other hydrocarbons, with implications for photochemical haze generation and habitability.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.16825-b31b1b.svg)](https://arxiv.org/abs/2508.16825) | **Technosignature Searches of Interstellar Objects**  |
|| J. R. A. Davenport, et al. -- incl., <mark>A. Frank</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| *17 pages, 5 figures. Submitted to journal for peer review, community feedback welcomed*|
|**Abstract**|            With the discovery of the third confirmed interstellar object (ISO), 3I/ATLAS, we have entered a new phase in the exploration of these long-predicted objects. Though confirmed discovery of ISOs is quite recent, their utility as targets in the search for technosignatures (historically known as the Search for Extraterrestrial Intelligence -- SETI) has been discussed for many decades. With the upcoming NSF-DOE Vera C. Rubin Observatory's Legacy Survey of Space and Time (LSST), the discovery and tracking of such objects is expected to become routine, and thus so must our examination of these objects for possible technosignatures. Here we review the literature surrounding ISOs as targets for technosignatures, which provides a well-developed motivation for such exploration. We outline four broad classes of technosignatures that are well suited for ISO follow-up, including the type of data needed and the best timing for study. Given the limitations in the current understanding of ISOs, we show that care must be taken in identifying technosignatures based primarily on comparison to objects in the Solar System. We therefore provide a roadmap for careful and consistent study of the population of ISOs in the hope of identifying technosignatures.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.17307-b31b1b.svg)](https://arxiv.org/abs/2508.17307) | **IGR J17091-3624: Newly Formed Periodic Dips Detected in the 2025 Outburst**  |
|| Z. Lin, et al. -- incl., <mark>Y. Wang</mark>, <mark>J. Liu</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| *16 pages, 10 figures, 1 tables; submitted*|
|**Abstract**|            The black hole low-mass X-ray binary (LMXB) candidate IGR J17091-3624 experienced a hard-state-only outburst in 2025. In this paper, we show that IXPE detected a series of intermittent X-ray dips, spanning a total interval of ~1 day. Subsequent observations with NICER, EP, NuSTAR, and Swift reveal that these dips recur with a period of 2.83$\pm$0.07 days and are accompanied by an increase in spectral hardness. This is the first time such quasi-periodic dipping behavior has been observed in this target since discovery. Our spectral analysis shows that the dips can be explained by obscuration from an ionized absorber characterized by an ionization parameter of $log{\xi}$ ~1-3 erg cm s$^{-1}$ and an equivalent hydrogen column density of $N^{\rm zxipcf}_{\rm H}$~(1-30)$\times10^{22}$ cm$^{-2}$. The periodic reappearance of the absorber is likely caused by obscuring material located in the outer accretion disk, modulated by the binary orbital period. If confirmed, this period would suggest that the donor star in IGR J17091-3624 has deviated from the standard main-sequence evolutionary path and is likely a (partially) stripped giant. In the optical band, no significant periodicity or correlation with the X-ray dips was detected, whereas the radio counterpart exhibited a flat to steep spectrum, in contrast to the inverted spectrum typically observed during the hard state of LMXBs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18097-b31b1b.svg)](https://arxiv.org/abs/2508.18097) | **A$^3$COSMOS: The dust content of massive quiescent galaxies and its evolution with cosmic time**  |
|| S. Adscheid, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| *Accepted for publication in A&A; 10 pages, 5 pages appendix, 10 figures; Acknowledgements can be found in the appendix*|
|**Abstract**|            We study the dust content of massive ($\log(M_*/M_{\odot})\geq10.8$) quiescent galaxies (QGs) at redshift $z=0.5-3$ to place constraints on the evolution of their cold interstellar medium (ISM), and thereby obtain insights on the processes of galaxy quenching throughout cosmic time. We use a robust sample of 458 colour-selected QGs covered by the A$^3$COSMOS+A$^3$GOODSS database to perform a stacking analysis in the $uv$-domain and measure their mean dust masses from their stacked submillimetre luminosities. We use the CIGALE SED-fitting code to obtain star formation histories and infer the time since quenching for all QGs in our sample. We use this information to gain insight on the time evolution of the dust content after quenching. Most QGs in our sample quenched around a redshift of $z\sim1.3$, following the peak of cosmic star formation. The majority of QGs observed at $z>1$ are recently quenched (i.e., quenched for no longer than $500\,$Myr), whereas the majority of QGs observed at $z<1$ have already been quenched for a significant amount of time ($\gtrsim1\,$Gyr). This implies that high-redshift galaxies ($z\gtrsim2$) are ideal for studying the mechanisms of quenching and its effects on the ISM, while lower-redshift galaxies are more suitable for studying the long-term effects of the QG environment on their ISM. We obtain upper limits on the dust mass fraction of the QG population, pointing towards lower dust content in high-redshift massive QGs than found by earlier stacking studies, and significantly lower (by a factor $\sim2-6$) than that of normal star forming galaxies. We also place constraints on the initial gas fraction right after quenching. We find that within the first $\sim600\,$Myr after quenching, QGs already lose on average $\gtrsim70\%$ of their cold ISM. Our findings support a gas consumption or removal scenario acting on short timescales.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.18126-b31b1b.svg)](https://arxiv.org/abs/2508.18126) | **Inferring Mbh-Mbulge Evolution from the Gravitational Wave Background**  |
|| C. Matt, et al. -- incl., <mark>J. Baier</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| **|
|**Abstract**|            We test the impact of an evolving supermassive black hole (SMBH) mass scaling relation (Mbh-Mbulge) on the predictions for the gravitational wave background (GWB). The observed GWB amplitude is 2-3 times higher than predicted by astrophysically informed models which suggests the need to revise the assumptions in those models. We compare a semi-analytic model's ability to reproduce the observed GWB spectrum with a static versus evolving-amplitude Mbh-Mbulge relation. We additionally consider the influence of the choice of galaxy stellar mass function on the modeled GWB spectra. Our models are able to reproduce the GWB amplitude with either a large number density of massive galaxies or a positively evolving Mbh-Mbulge amplitude (i.e., the Mbh / Mbulge ratio was higher in the past). If we assume that the Mbh-Mbulge amplitude does not evolve, our models require a galaxy stellar mass function that implies an undetected population of massive galaxies (Mstellar > 10^11 Msun at z > 1). When the Mbh-Mbulge amplitude is allowed to evolve, we can model the GWB spectrum with all fiducial values and an Mbh-Mbulge amplitude that evolves as alpha(z) = alpha_0 (1 + z)^(1.04 +/- 0.5).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2508.16951-b31b1b.svg)](https://arxiv.org/abs/2508.16951) | **MIDIS: Unveiling the Star Formation History in massive galaxies at $1<z<4.5$ with spectro-photometric analysis**  |
|| M. Annunziatella, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-08-26*|
|*Comments*| *accepted for publication in A&A*|
|**Abstract**|            We investigate the star formation histories (SFHs) of a sample of massive galaxies ($M_\star \geq 10^{10} \, M_\odot$) in the redshift range $1 < z < 4.5$. We analyze spectro-photometric data combining broadband photometry from HST and JWST with low-resolution grism spectroscopy from JWST/NIRISS, obtained as part of the MIDIS (MIRI Deep Imaging Survey) program. SFHs are derived through spectral energy distribution (SED) fitting using two independent codes, BAGPIPES and Synthesizer, under various SFH assumptions. This approach enables a comprehensive assessment of the biases introduced by different modeling choices. The inclusion of NIRISS spectroscopy, even with its low resolution, significantly improves constraints on key physical parameters, such as the mass-weighted stellar age ($t_M$) and formation redshift ($z_{\mathrm{form}}$), by narrowing their posterior distributions. The massive galaxies in our sample exhibit rapid stellar mass assembly, forming 50\% of their mass between $3 \leq z \leq 9$. The highest inferred formation redshifts are compatible with elevated star formation efficiencies ($\epsilon$) at early epochs. Non-parametric SFHs generally imply an earlier and slower mass assembly compared to parametric forms, highlighting the sensitivity of inferred formation timescales to the chosen SFH model, particularly for galaxies at $z < 2$. Quiescent galaxies are, on average, older ($t_M \sim 1.1$ Gyr) and assembled more rapidly at earlier times than their star-forming counterparts. These findings support the ``downsizing'' scenario, in which more massive and passive systems form earlier and more efficiently.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2508.16775/./figures/fig_SwiftJ1910_I_BESS_1.2arcmin_FORS2.png', 'tmp_2508.16775/./figures/fig_j1910_smarts_wfc_sdssr_GLS_n1_n2_n3_NEW.png', 'tmp_2508.16775/./figures/fig_j1910_Ha_osiris_fors2_period_vs_Gauss_sep.png']
copying  tmp_2508.16775/./figures/fig_SwiftJ1910_I_BESS_1.2arcmin_FORS2.png to _build/html/
copying  tmp_2508.16775/./figures/fig_j1910_smarts_wfc_sdssr_GLS_n1_n2_n3_NEW.png to _build/html/
copying  tmp_2508.16775/./figures/fig_j1910_Ha_osiris_fors2_period_vs_Gauss_sep.png to _build/html/
exported in  _build/html/2508.16775.md
    + _build/html/tmp_2508.16775/./figures/fig_SwiftJ1910_I_BESS_1.2arcmin_FORS2.png
    + _build/html/tmp_2508.16775/./figures/fig_j1910_smarts_wfc_sdssr_GLS_n1_n2_n3_NEW.png
    + _build/html/tmp_2508.16775/./figures/fig_j1910_Ha_osiris_fors2_period_vs_Gauss_sep.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Ha}{H\textalpha}$
$\newcommand{\Hb}{H\textbeta}$
$\newcommand{\Hg}{H\textgamma}$
$\newcommand{\Hd}{H\textdelta}$
$\newcommand{\Hep}{H\textepsilon}$
$\newcommand{\Ion}[2]{#1 {\sc #2}}$
$\newcommand{\Line}[3]{#1 {\sc #2}~\textlambda #3}$
$\newcommand{\kms}{\mbox{\mathrm{km~s^{-1}}}}$
$\newcommand{\he}[1]$
$\newcommand{\hel}[2]$
$\newcommand{\Msun}{\mbox{M_{\rm \odot}}}$
$\newcommand{\Rsun}{\mbox{R_{\rm \odot}}}$
$\newcommand{\sw}{\textit{Swift} J1910.2--0546}$
$\newcommand{\jj}{J1910}$
$\newcommand{\porb}{P}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\ebvval}[1]{E(B-V) = #1}$
$\newcommand{\ergs}{erg s^{-1} cm^{-2}}$
$\newcommand{\savefootnote}[2]{\footnote{\label{#1}#2}}$
$\newcommand{\repeatfootnote}[1]{\textsuperscript{\ref{#1}}}$</div>



<div id="title">

# Characterising the short-orbital period X-ray transient $\sw$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2508.16775-b31b1b.svg)](https://arxiv.org/abs/2508.16775)<mark>Appeared on: 2025-08-26</mark> -  _13 pages, 10 figures. Accepted for publication in A&A_

</div>
<div id="authors">

J. M. Corral-Santana, et al. -- incl., <mark>A. P. Garcia</mark>

</div>
<div id="abstract">

**Abstract:** $\sw$ (=MAXI J1910 $-$ 057) is a Galactic X-ray transient discovered during a bright outburst in 2012. Its X-ray spectral and timing properties point to a black-hole accretor, yet the orbital period remains uncertain, and no reliable dynamical constraints on the binary parameters are available.The 2012 event, extensively monitored at X-ray and optical wavelengths, offers a rare opportunity to investigate the structure and dynamics of the system and to constrain its fundamental properties. We use time-series optical photometry and spectroscopy, obtained during outburst and quiescence, to estimate the orbital period, characterise the donor star, determine the interstellar extinction, distance, and system geometry, and constrain the component masses. Multi-site $r$ -band and clear-filter light curves and WHT/ACAM spectra from the 2012 outburst are combined with time-series spectroscopy from GTC/OSIRIS and VLT/FORS2 in quiescence.  Period searches are conducted using generalised Lomb–Scargle, phase-dispersion minimisation, and analysis-of-variance algorithms. Diffuse interstellar bands constrain $E(B\!-\!V)$ , while empirical correlations involving $\Ha$ yield estimates of $K_2$ , $q$ , and $i$ . We detect a coherent, double-humped modulation with a period of $0.0941\pm0.0007$ d ( $2.26\pm0.02$ h) during the outburst. Its morphology is consistent with an early superhump, suggesting that the true orbital period may be slightly shorter than 4.52 h. The $\Ha$ radial velocity curves do not yield a definitive orbital period. In quiescence, TiO bands indicate an M3--M3.5 donor contributing $\simeq\!70\%$ of the red continuum. Diffuse interstellar bands give $E(B\!-\!V)=0.60\pm0.05$ and $N_{\mathrm{H}}=(3.9\pm1.3)\times10^{21}$ cm $^{-2}$ , placing the system at a distance of 2.8–4.0 kpc. The $\Ha$ line width in quiescence ( $\mathrm{FWHM}_0 = 990\pm45$ $\kms$ ), via a FWHM- $K_2$ calibration, provides an estimate of $K_2$ , while its double-peaked profile gives $q$ and the orbital inclination. The latter appears much higher than estimates from X-ray studies. Adopting the resulting $K_{2}=230\pm17$ $\kms$ and $q=0.032\pm0.010$ , and two orbital period scenarios (2.25 and 4.50 \; h), Monte Carlo sampling returns a compact object mass $M_{1}=8$ -- $11$ $\Msun$ and an inclination $i = 13^{\circ}$ – $18^{\circ}$ for plausible donor masses ( $M_{2}=0.25$ -- $0.35$ $\Msun$ ). We favour an orbital period of 4.5 \; h. $\sw$ may be a short-period, low-inclination black hole X-ray transient, although a neutron star accretor cannot be completely ruled out. Further phase-resolved spectroscopy and photometry during quiescence are needed to better determine its fundamental parameters.

</div>

<div id="div_fig1">

<img src="tmp_2508.16775/./figures/fig_SwiftJ1910_I_BESS_1.2arcmin_FORS2.png" alt="Fig1" width="100%"/>

**Figure 1. -** VLT/FORS2 $I$-band acquisition image obtained during quiescence. $\jj$ is marked with a red cross at the centre of the 1.2$\arcmin$ field. North is up, east is to the left.
     (*fig:fc*)

</div>
<div id="div_fig2">

<img src="tmp_2508.16775/./figures/fig_j1910_smarts_wfc_sdssr_GLS_n1_n2_n3_NEW.png" alt="Fig7" width="100%"/>

**Figure 7. -** Top panel: $r$-band light curves from SMARTS 0.9-m and INT/WFC, expressed as flux ratios and magnitudes, relative to the comparison star. Middle panels: SMARTS light curve (top left) and INT/WFC light curves from the first three nights when $\jj$ maintained an almost constant flux level, after subtracting the nightly averages. The red dashed curve represents the best-fit sine wave with a period corresponding to the highest peak in the periodogram shown below. Bottom left panel: GLS periodogram of the four $r$-band light curves displayed in the middle panels (4, 9, 10, and 11 Jun 2012). The highest peak corresponds to a period of $0.0941$\;d ($= 2.26$\;h). Bottom middle and right panels: The four light curves folded on this period and twice that value. The pale blue points represent the unbinned data, while the purple points are binned data across 40 phase intervals. Zero phase corresponds to the HJD of the first data point, and a full cycle is repeated for continuity. (*j1910_mnras_fig_GLS_01*)

</div>
<div id="div_fig3">

<img src="tmp_2508.16775/./figures/fig_j1910_Ha_osiris_fors2_period_vs_Gauss_sep.png" alt="Fig3" width="100%"/>

**Figure 3. -** Top panel: Best sine-fit periods derived from the radial velocity curves of the $\Ha$ emission line in the OSIRIS spectra, as a function of the double-Gaussian separation used in the velocity extraction. The shaded horizontal band indicates the periods derived from the light curves. Error bars represent 1$\sigma$ uncertainties. Bottom panel: Periods corresponding to the highest peaks in the GLS (blue circles) and AOV (orange squares) periodograms derived from the quiescent FORS2 $\Ha$ radial velocity curves, as a function of the Gaussian separation used to extract the velocities. The period search was restricted to the range 0.068--0.25\;d. The lower shaded horizontal band marks the periods observed in the photometric light curves, while the upper band corresponds to twice these values. (*fig:j1910_mnras_fig_Ha_RVC*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2508.16775"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

127  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
